# Corner Extraction Approach

Try the chistyakov 'new-simple-fix' approach: extract smaller layouts from larger ones by looking at corners.

In [ ]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import json

getcontext().prec = 30
scale_factor = 1

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (float(Decimal('0.0') * scale_factor), float(tip_y * scale_factor)),
            (float(top_w / Decimal('2') * scale_factor), float(tier_1_y * scale_factor)),
            (float(top_w / Decimal('4') * scale_factor), float(tier_1_y * scale_factor)),
            (float(mid_w / Decimal('2') * scale_factor), float(tier_2_y * scale_factor)),
            (float(mid_w / Decimal('4') * scale_factor), float(tier_2_y * scale_factor)),
            (float(base_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(base_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(mid_w / Decimal('4')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(mid_w / Decimal('2')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(top_w / Decimal('4')) * scale_factor), float(tier_1_y * scale_factor)),
            (float(-(top_w / Decimal('2')) * scale_factor), float(tier_1_y * scale_factor)),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

def get_tree_list_side_length(tree_list):
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def load_solution(csv_path):
    df = pd.read_csv(csv_path)
    df['x'] = df['x'].astype(str).str.strip().str.lstrip('s')
    df['y'] = df['y'].astype(str).str.strip().str.lstrip('s')
    df['deg'] = df['deg'].astype(str).str.strip().str.lstrip('s')
    df[['group_id', 'item_id']] = df['id'].str.split('_', n=2, expand=True)
    
    solution = {}
    for group_id, group_data in df.groupby('group_id'):
        n = int(group_id)
        trees = [(row['x'], row['y'], row['deg']) for _, row in group_data.iterrows()]
        solution[n] = trees
    
    return solution

def score_config(trees_data):
    tree_list = [ChristmasTree(x, y, deg) for x, y, deg in trees_data]
    side = get_tree_list_side_length(tree_list)
    n = len(trees_data)
    return side ** 2 / n

print('Functions defined.')

In [ ]:
# Load current best solution
solution = load_solution('/home/code/experiments/004_cpp_sa_optimizer/input.csv')
print(f'Loaded {len(solution)} N configurations')

# Calculate current scores
current_scores = {n: score_config(solution[n]) for n in range(1, 201)}
total_score = sum(current_scores.values())
print(f'Current total score: {total_score:.6f}')

In [ ]:
def extract_corner_subset(trees_data, target_n, corner='top_left'):
    """
    Extract target_n trees closest to a specific corner from a larger configuration.
    
    corners: 'top_left', 'top_right', 'bottom_left', 'bottom_right'
    """
    # Get tree positions
    positions = [(float(x), float(y), deg) for x, y, deg in trees_data]
    
    # Calculate bounding box
    xs = [p[0] for p in positions]
    ys = [p[1] for p in positions]
    min_x, max_x = min(xs), max(xs)
    min_y, max_y = min(ys), max(ys)
    
    # Define corner point
    if corner == 'top_left':
        cx, cy = min_x, max_y
    elif corner == 'top_right':
        cx, cy = max_x, max_y
    elif corner == 'bottom_left':
        cx, cy = min_x, min_y
    elif corner == 'bottom_right':
        cx, cy = max_x, min_y
    else:
        raise ValueError(f'Unknown corner: {corner}')
    
    # Calculate distance to corner for each tree
    distances = []
    for i, (x, y, deg) in enumerate(positions):
        dist = np.sqrt((x - cx)**2 + (y - cy)**2)
        distances.append((dist, i, x, y, deg))
    
    # Sort by distance and take closest target_n trees
    distances.sort()
    closest = distances[:target_n]
    
    # Extract the subset
    subset = [(str(x), str(y), str(deg)) for _, _, x, y, deg in closest]
    
    return subset

print('Corner extraction function defined.')

In [ ]:
# Try corner extraction for small N values
improvements = []
corners = ['top_left', 'top_right', 'bottom_left', 'bottom_right']

print('Trying corner extraction...')
for target_n in range(2, 51):  # Focus on small N
    current_score = current_scores[target_n]
    best_score = current_score
    best_config = solution[target_n]
    best_source = None
    
    # Try extracting from larger N configurations
    for source_n in range(target_n + 1, 201):
        for corner in corners:
            subset = extract_corner_subset(solution[source_n], target_n, corner)
            subset_score = score_config(subset)
            
            if subset_score < best_score - 1e-9:
                best_score = subset_score
                best_config = subset
                best_source = (source_n, corner)
    
    if best_source is not None:
        improvements.append((target_n, current_score, best_score, current_score - best_score, best_source))
        solution[target_n] = best_config
        print(f'  N={target_n:3d}: {current_score:.6f} -> {best_score:.6f} (from N={best_source[0]}, {best_source[1]})')

print(f'\nFound {len(improvements)} improvements')
total_improvement = sum(delta for _, _, _, delta, _ in improvements)
print(f'Total improvement: {total_improvement:.9f}')

In [ ]:
# Calculate new total score
new_total = sum(score_config(solution[n]) for n in range(1, 201))

print(f'Original score: {total_score:.6f}')
print(f'After corner extraction: {new_total:.6f}')
print(f'Improvement: {total_score - new_total:.9f}')
print(f'Target: 68.919154')
print(f'Gap to target: {new_total - 68.919154:.6f}')

In [ ]:
# Save submission
rows = []
for n in range(1, 201):
    for i, (x, y, deg) in enumerate(solution[n]):
        rows.append({
            'id': f'{n:03d}_{i}',
            'x': f's{x}',
            'y': f's{y}',
            'deg': f's{deg}'
        })

submission_df = pd.DataFrame(rows)
submission_df.to_csv('/home/submission/submission.csv', index=False)
submission_df.to_csv('submission.csv', index=False)
print(f'Saved submission with {len(submission_df)} rows')

In [ ]:
# Save metrics
metrics = {
    'cv_score': new_total,
    'original_score': total_score,
    'improvement': total_score - new_total,
    'num_improvements': len(improvements),
    'cpp_optimizer_result': 'No improvements found after 4 generations',
    'corner_extraction_result': f'{len(improvements)} improvements found'
}

with open('metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f'Saved metrics: {metrics}')